In [1]:
import tensorflow as tf
from tqdm import tqdm
import numpy as np
tf.keras.backend.set_floatx('float16')

In [2]:
from data_management.VOC2012ManagerObjDetection import VOC2012ManagerObjDetection
db_manager = VOC2012ManagerObjDetection("C:/Users/antho/Documents/GitHub/data/VOCdevkit/VOC2012/",
                                        batch_size=32, floatType=32)

In [3]:
from models.SSD300 import SSD300
SSD300_model = SSD300(21, floatType=16)
SSD300_model.build(input_shape=(32, 300, 300, 3))
SSD300_model.load_vgg16_imagenet_weights()

In [4]:
optimizer = tf.keras.optimizers.SGD(
    learning_rate=10**-3,
    momentum=0.9)

In [5]:
from data_management.preprocessing import *
# Preprocess data
#saveGTdata("C:/Users/antho/Documents/GitHub/data/VOCdevkit/VOC2012/",
#           "C:/Users/antho/Documents/GitHub/test/")

In [6]:
from data_management.preprocessing import *
imgs, confs, locs = loadGTdata("C:/Users/antho/Documents/GitHub/preprocessed_data/")

  1%|▉                                                                                 | 4/374 [00:00<00:09, 38.81it/s]

Loading images...


  8%|██████▏                                                                         | 29/374 [00:00<00:01, 282.32it/s]

Loading confs gt...


  5%|████                                                                            | 19/374 [00:00<00:01, 184.97it/s]

Loading locs gt...


100%|███████████████████████████████████████████████████████████████████████████████| 374/374 [00:03<00:00, 118.01it/s]


In [7]:
# if needed
#SSD300_model.load_weights(path)

In [ ]:
from utils.train import *
train(SSD300_model, optimizer, db_manager, imgs, confs, locs, weights_path="../weights/",
          num_epoch=250, inter_save=1)